In [1]:
import requests
from bs4 import BeautifulSoup
from nba_api.stats.endpoints import leaguedashteamstats, leaguedashteamshotlocations
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [52]:
# Create an instance of the LeagueDashTeamStats endpoint
team_shot_locations_connect = leaguedashteamshotlocations.LeagueDashTeamShotLocations()

# Retrieve the data
team_shot_locations_data = team_shot_locations_connect.get_data_frames()[0]  # Get the first DataFrame

team_shot_locations = pd.DataFrame(team_shot_locations_data)
team_shot_locations = team_shot_locations

teams = team_shot_locations['']['TEAM_NAME']

rim_data = team_shot_locations[['Restricted Area']]
rim_data = rim_data['Restricted Area'][['FGM', 'FGA', 'FG_PCT']]

short_mid_data = team_shot_locations[['In The Paint (Non-RA)']]
short_mid_data = short_mid_data['In The Paint (Non-RA)'][['FGM', 'FGA', 'FG_PCT']]

long_mid_data = team_shot_locations[['Mid-Range']]
long_mid_data = long_mid_data['Mid-Range'][['FGM', 'FGA', 'FG_PCT']]

corner_3_data = team_shot_locations[['Corner 3']]
corner_3_data = corner_3_data['Corner 3'][['FGM', 'FGA', 'FG_PCT']]

above_the_break_data = team_shot_locations[['Above the Break 3']]
above_the_break_data = above_the_break_data['Above the Break 3'][['FGM', 'FGA', 'FG_PCT']]

rim = pd.concat([teams, rim_data], axis=1).set_index('TEAM_NAME')
short_mid = pd.concat([teams, short_mid_data], axis=1).set_index('TEAM_NAME')
long_mid = pd.concat([teams, long_mid_data], axis=1).set_index('TEAM_NAME')
corner_3 = pd.concat([teams, corner_3_data], axis=1).set_index('TEAM_NAME')
above_the_break = pd.concat([teams, above_the_break_data], axis=1).set_index('TEAM_NAME')

above_the_break

,FGM,FGA,FG_PCT
TEAM_NAME,,,
Atlanta Hawks,244,650,0.375
Boston Celtics,279,787,0.355
Brooklyn Nets,230,646,0.356
Charlotte Hornets,186,492,0.378
Chicago Bulls,226,636,0.355
Cleveland Cavaliers,214,629,0.340
Dallas Mavericks,272,716,0.380
Denver Nuggets,231,632,0.366
Detroit Pistons,178,527,0.338


In [9]:
team_shot_locations.columns

MultiIndex([(                     '',   'TEAM_ID'),
            (                     '', 'TEAM_NAME'),
            (      'Restricted Area',       'FGM'),
            (      'Restricted Area',       'FGA'),
            (      'Restricted Area',    'FG_PCT'),
            ('In The Paint (Non-RA)',       'FGM'),
            ('In The Paint (Non-RA)',       'FGA'),
            ('In The Paint (Non-RA)',    'FG_PCT'),
            (            'Mid-Range',       'FGM'),
            (            'Mid-Range',       'FGA'),
            (            'Mid-Range',    'FG_PCT'),
            (        'Left Corner 3',       'FGM'),
            (        'Left Corner 3',       'FGA'),
            (        'Left Corner 3',    'FG_PCT'),
            (       'Right Corner 3',       'FGM'),
            (       'Right Corner 3',       'FGA'),
            (       'Right Corner 3',    'FG_PCT'),
            (    'Above the Break 3',       'FGM'),
            (    'Above the Break 3',       'FGA'),
            

In [5]:
# Create an instance of the LeagueDashTeamStats endpoint
team_stats = leaguedashteamstats.LeagueDashTeamStats()

# Retrieve the data
team_stats_data = team_stats.get_data_frames()[0]  # Get the first DataFrame

# Grab the PPP necessary stats
ovr_PPP_data = team_stats_data[['TEAM_NAME', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'TOV']]
PPP_stats = ovr_PPP_data.copy()

# Calculate points for each team
PPP_stats['FG2_POINTS'] = (PPP_stats['FGM'] - PPP_stats['FG3M']) * 2
PPP_stats['FG3_POINTS'] = PPP_stats['FG3M'] * 3
PPP_stats['FT_POINTS'] = PPP_stats['FTM']

# Calculate total points scored by each team
PPP_stats['TOTAL_POINTS'] = PPP_stats['FG2_POINTS'] + PPP_stats['FG3_POINTS'] + PPP_stats['FT_POINTS']
PPP_stats = PPP_stats.set_index('TEAM_NAME')

# Calculate totals for all stats including points
totals = PPP_stats.select_dtypes(np.number).sum().rename('League Totals')
PPP_stats = pd.concat([PPP_stats, totals.to_frame().T])

# Calculate Points per Possession (PPP) for each team
total_poss = (PPP_stats['FGA'] + (0.44 * PPP_stats['FTA']) + PPP_stats['TOV'])
PPP_stats['PPP'] = round((PPP_stats['TOTAL_POINTS'] / (PPP_stats['FGA'] + (0.44 * PPP_stats['FTA']) + PPP_stats['TOV'])),2)
PPP_stats[['PPP']].sort_values('PPP', ascending=False)

,PPP
Indiana Pacers,1.09
Milwaukee Bucks,1.08
Oklahoma City Thunder,1.07
Philadelphia 76ers,1.06
Dallas Mavericks,1.06
Boston Celtics,1.05
Atlanta Hawks,1.03
Phoenix Suns,1.03
Miami Heat,1.03
Brooklyn Nets,1.03


In [67]:
# Create an instance of the LeagueDashTeamStats endpoint
team_stats = synergyplaytypes.SynergyPlayTypes()

# Retrieve the data
team_stats_data = team_stats.get_data_frames() # Get the first DataFrame

In [68]:
team_stats_data

[Empty DataFrame
 Columns: [SEASON_ID, TEAM_ID, TEAM_ABBREVIATION, TEAM_NAME, PLAY_TYPE, TYPE_GROUPING, PERCENTILE, GP, POSS_PCT, PPP, FG_PCT, FT_POSS_PCT, TOV_POSS_PCT, SF_POSS_PCT, PLUSONE_POSS_PCT, SCORE_POSS_PCT, EFG_PCT, POSS, PTS, FGM, FGA, FGMX]
 Index: []]